## Task 1: Bike Rental Forecasting – Project Overview

The goal of this project is to develop and evaluate regression models that predict the daily number of bike rentals based on weather, calendar, and seasonal data. The dataset comes from the Capital Bikeshare system in Washington, D.C., and is publicly available via the UCI repository.

In this notebook, we will:
1. Load and explore the dataset, identifying key variables and checking for missing values or duplicates.
2. Perform exploratory data analysis (EDA) to understand relationships between input features and the target variable (`cnt` – total count of rentals).
3. Prepare the data through encoding, scaling, and splitting into training and testing sets.
4. Build and evaluate multiple regression models (Linear Regression, Ridge, Lasso, Decision Trees, Random Forest, Gradient Boosting).
5. Assess model performance using metrics such as MAE, MSE, RMSE, and R².
6. Interpret feature importance and draw conclusions about the key factors influencing rental demand.
7. Summarize findings and reflect on possible improvements and future directions.

The final goal is to find the most accurate model and understand which features have the strongest impact on rental predictions.
